In [1]:
import os
import pandas as pd
import re
import subprocess
from collections import defaultdict


In [2]:

def digest_move_summary(move_lines):

    mode = 'koulu' # koulu - luokka - ryhma - laji
    school_name = ''
    luokka_list = []
    laji_id = 'x'
    scores = defaultdict(list)
    sc_units = dict()

    for txt in move_lines:
        if (mode == 'koulu'):
            r = re.search(r"^Tulosten yhteenveto: (.*)$",txt)
            if (r):
                school_name = r.group(1)
                mode = 'luokka'

        if (mode == 'luokka'):
            r = re.search(r"\s*([0-9]+)\. luokka.*$",txt)
            if (r):
                luokka_list.append(r.group(1))

            r = re.search(r"\s*OSIO",txt)
            if (r):
                mode = 'ryhma'

        if (mode == 'ryhma' or mode == 'laji'):
            r = re.search(r"\s*(.*) \((.*)\)",txt)
            if r and not (('median' in txt or ('medel' in txt))):
                mode = 'laji'
                laji_id = r.group(1)
                sc_units[laji_id] = r.group(2)

        if (mode == 'laji'):
            r = re.search(r"\s*(\d\S*)\s*$",txt)
            if r and len(scores[laji_id]) < (len(luokka_list)*2):
                scores[laji_id].append(r.group(1))
                
    scorefields = []
    for k in scores.keys():
        for i,v in enumerate(scores[k]):
            scorefields.append((school_name,
                  k,
                  sc_units[k],
                  luokka_list[int(i/2)],
                              'p' if i%2 else 't',
                              v))
    df = pd.DataFrame(scorefields,
                      columns=['koulun_nimi','osio','yks','luokka',
                               'sukupuoli','move_tulos'])
    
    return df




In [4]:
"asdsdad".index('d')

2

In [5]:

def extract_text_cells(tab_txt):

    viiva_rows = tab_txt.split('\n')
    textcells = []

    for rowid,row_in in enumerate(viiva_rows):
        col = 0
        m = True
        while (m):
            m = re.search("^(\s*)(\S+)(.*)$",row_in[col:])
            if m:
                col += len(m.group(1)) # whitespace
                mtxt = m.group(2)
                if ("%" in mtxt):
                    mtxt = mtxt[0:(mtxt.index('%')+1)]
                textcells.append((rowid,col,mtxt))
                col += len(mtxt)

    return textcells



In [6]:
# init

kouluid_map = pd.DataFrame(pd.read_excel("../input/SALAINEN_Koulu_KouluID.xlsx"),columns=['Koulu_ID','Koulu']).set_index('Koulu').Koulu_ID

# main loop

file_name = "../../m-suojaamaton/movetulokset/aleksis_kiven_koulu_03380_move2022.pdf"
file_name = "../../m-suojaamaton/movetulokset/arabian_peruskoulu_03743_move2022.pdf"



In [7]:
# summary page results

result = subprocess.run(["pdftotext -f 7 -l 7 "+file_name+" -"], shell=True, capture_output=True, text=True)
move_summarypage = result.stdout
summary_df = digest_move_summary(move_summarypage.split("\n"))
summary_df


,koulun_nimi,osio,yks,luokka,sukupuoli,move_tulos
0,Arabian peruskoulu,20 metrin viivajuoksu,"mediaani, min:sek",5,t,04:26
1,Arabian peruskoulu,20 metrin viivajuoksu,"mediaani, min:sek",5,p,04:22
2,Arabian peruskoulu,20 metrin viivajuoksu,"mediaani, min:sek",8,t,07:22
3,Arabian peruskoulu,20 metrin viivajuoksu,"mediaani, min:sek",8,p,03:40
4,Arabian peruskoulu,Vauhditon 5-loikka,"mediaani, metriä",5,t,7.8
5,Arabian peruskoulu,Vauhditon 5-loikka,"mediaani, metriä",5,p,8.25
6,Arabian peruskoulu,Vauhditon 5-loikka,"mediaani, metriä",8,t,9.9
7,Arabian peruskoulu,Vauhditon 5-loikka,"mediaani, metriä",8,p,8.5
8,Arabian peruskoulu,Heitto-kiinniottoyhdistelmä,"mediaani, krt",5,t,13
9,Arabian peruskoulu,Heitto-kiinniottoyhdistelmä,"mediaani, krt",5,p,9


In [8]:
# Saving summary page

# Extract school metadata

m = re.search(r".*_move(20\d\d)\.",file_name)
yearid = 1984
if m:
    yearid = int(m.group(1))

schoolname = summary_df.iloc[0].koulun_nimi
schoolid=-1

schoolnames = [schoolname,schoolname.replace('koulu','peruskoulu')]
for n in schoolnames:
    if n in kouluid_map.index:
        schoolid = kouluid_map.loc[n]
schoolid

csvbasename = "move_%04d_koulu_%d" % (yearid,schoolid)
print("Base ID %s" % csvbasename)

summary_df['Koulu_ID'] = schoolid
summary_df = pd.DataFrame(summary_df,columns=[(summary_df.columns[-1])] + list(summary_df.columns[0:-1]))
csvname = "%s_summary.tsv"%csvbasename
summary_df.to_csv(csvname,sep='\t')

print("Written %s"%csvname)

Base ID move_2022_koulu_102
Written move_2022_koulu_102_summary.tsv


In [9]:
# tertile (osuus) report pages

result = subprocess.run(["pdftotext -f 8 -fixed 4 "+file_name+" -"], shell=True, capture_output=True, text=True)
move_details = result.stdout
osuus_pgs = [pg for pg in move_details.split('\f') if ', osuudet ' in pg]

In [22]:
#print(osuus_pgs[1])

In [21]:
for osuus_txt in osuus_pgs:
    luokka = "-"
    m = re.search(r"(\d)\.?\s+luokka",osuus_txt)
    if m:
        luokka = m.group(1)

    laji = "-"
    m = re.search(r"\s*(\S.*\S), osuudet",osuus_txt)
    if m:
        laji = m.group(1)

    #print(luokka, laji)
    
    
    if '20 m viivajuoksu' == laji:
        cells = extract_text_cells(osuus_txt)
        colvals = [(c,t) for (r,c,t) in cells if t.endswith('%') and r>2]
        barchart_values = pd.DataFrame(colvals,columns=['xpos','value']).sort_values('xpos').reset_index(drop=True)
        #print(barchart_values)
        
        barnames = [("osuus_%s"%x) for x in ['ref_A','A','ref_B','B','ref_C','C']]
        osuus1_df = pd.DataFrame([[schoolname,laji,luokka,'t']+list(barchart_values.value[0:6])])
        osuus1_df.columns = ['koulun_nimi','laji','luokka','sukupuoli']+barnames
        osuus1_df

        osuus2_df = pd.DataFrame([[schoolname,laji,luokka,'p']+list(barchart_values.value[6:12])])
        osuus2_df.columns = ['koulun_nimi','laji','luokka','sukupuoli']+barnames
        osuus2_df

        osuus_df = pd.concat([osuus1_df,osuus2_df])
        osuus_df['Koulu_ID'] = schoolid
        osuus_df = pd.DataFrame(osuus_df,columns=['Koulu_ID']+list(osuus1_df.columns)).reset_index(drop=True)
        print(osuus_df)
        
        
        

   Koulu_ID         koulun_nimi              laji luokka sukupuoli osuus_A  \
0       102  Arabian peruskoulu  20 m viivajuoksu      5         t   40.3%   
1       102  Arabian peruskoulu  20 m viivajuoksu      5         p   41.1%   

  osuus_ref_A osuus_B osuus_ref_B osuus_C osuus_ref_C  
0       21.4%   27.6%       32.1%   32.1%       46.4%  
1       24.3%     27%       40.5%   31.9%       35.1%  
   Koulu_ID         koulun_nimi              laji luokka sukupuoli osuus_A  \
0       102  Arabian peruskoulu  20 m viivajuoksu      8         t   33.2%   
1       102  Arabian peruskoulu  20 m viivajuoksu      8         p   35.5%   

  osuus_ref_A osuus_B osuus_ref_B osuus_C osuus_ref_C  
0       35.7%   31.9%       35.7%   34.9%       28.6%  
1        8.7%   28.9%       30.4%   35.6%       60.9%  


In [16]:
osuus1_df

,koulun_nimi,laji,luokka,sukupuoli,osuus_A,osuus_ref_A,osuus_B,osuus_ref_B,osuus_C,osuus_ref_C
0,Arabian peruskoulu,20 m viivajuoksu,8,t,33.2%,35.7%,31.9%,35.7%,34.9%,28.6%


,Koulu_ID,koulun_nimi,laji,luokka,sukupuoli,osuus_A,osuus_ref_A,osuus_B,osuus_ref_B,osuus_C,osuus_ref_C
0,102,Arabian peruskoulu,20 m viivajuoksu,5,t,40.3%,21.4%,27.6%,32.1%,32.1%,46.4%
1,102,Arabian peruskoulu,20 m viivajuoksu,5,p,41.1%,24.3%,27%,40.5%,31.9%,35.1%


In [32]:
csvname = "%s_osuudet.tsv"%csvbasename
osuus_df.to_csv(csvname,sep='\t')

print("Written %s"%csvname)

Written move_2022_koulu_102_osuudet.tsv
